In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/Uni Leipzig/SS2022/bdlt_contrastive/dataset'

In [ ]:
%pip install datasets

In [ ]:
import re
import datasets
import os
import pandas as pd
import numpy as np

In [ ]:
def para(row, anchor_col="sentence1", para_col="sentence2", label_col="label", path="paws", name="small_diverse", split="train"):
  out_row = {
             "sentence1": row[anchor_col],
             "sentence2": row[para_col],
             "label": row[label_col],
             "path": path,
             "name": name,
             "split": split}
  return out_row

dataset_definitions = [
    {"path": "paws", "name": "labeled_final"},
    {"path": "paws", "name": "labeled_swap"},
    {"path": "paws", "name": "unlabeled_final"}]

for definition in dataset_definitions:
    # load max 50000 samples, only affects "unlabeled_final", all others are smaller
    dataset = datasets.load_dataset(path=definition["path"], name=definition["name"]).filter(lambda example: example["id"] <= 50000)
    for split in ["train", "validation", "test"]:
        out = []
        if split not in dataset:
            continue
        dataset_part = pd.DataFrame(dataset[split])
        #dataset_post = dataset_part.map(lambda x: para(model, tokenizer, x), num_proc=1)
        for idx, row in dataset_part.iterrows():
            out.append(para(row, split=split, path=definition["path"], name=definition["name"]))
        df = pd.DataFrame(out)
        #df = df[df["original_label"] == 1]
        df = df.reset_index()   
        df.to_csv(f'raw/out_{definition["path"]}_{definition["name"]}_{split}.csv', index=False)

